In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import time
import os

# Set random seed for reproducibility
tf.random.set_seed(42)

# Create a directory for saving generated images
if not os.path.exists('generated_images'):
    os.makedirs('generated_images')

# Load and preprocess the Fashion-MNIST dataset
print("Loading Fashion-MNIST dataset...")
fashion_mnist = keras.datasets.fashion_mnist
(train_images, _), (_, _) = fashion_mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  # Normalize images to [-1, 1]

BUFFER_SIZE = 60000
BATCH_SIZE = 256

# Create tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# DCGAN Generator
def make_generator_model():
    model = keras.Sequential([
        layers.Dense(7*7*256, use_bias=False, input_shape=(100,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        
        layers.Reshape((7, 7, 256)),
        layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        
        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        
        layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model

# DCGAN Discriminator
def make_discriminator_model():
    model = keras.Sequential([
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        
        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        
        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

# Simple Diffusion Model
def make_diffusion_model():
    model = keras.Sequential([
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=[28, 28, 1]),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.Conv2DTranspose(64, (3, 3), strides=2, activation='relu', padding='same'),
        layers.Conv2D(1, (3, 3), activation='tanh', padding='same')
    ])
    return model

# Loss functions
cross_entropy = keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Optimizers
generator_optimizer = keras.optimizers.Adam(1e-4)
discriminator_optimizer = keras.optimizers.Adam(1e-4)
diffusion_optimizer = keras.optimizers.Adam(1e-3)

# Create models
generator = make_generator_model()
discriminator = make_discriminator_model()
diffusion_model = make_diffusion_model()

# Training steps
@tf.function
def train_step_gan(images):
    noise = tf.random.normal([BATCH_SIZE, 100])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

@tf.function
def train_step_diffusion(images):
    noise = tf.random.normal(shape=images.shape)
    noisy_images = images + 0.1 * noise

    with tf.GradientTape() as tape:
        reconstructed = diffusion_model(noisy_images, training=True)
        loss = tf.reduce_mean(tf.square(images - reconstructed))

    gradients = tape.gradient(loss, diffusion_model.trainable_variables)
    diffusion_optimizer.apply_gradients(zip(gradients, diffusion_model.trainable_variables))

    return loss

# Training loops
def train_gan(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        gen_loss_list = []
        disc_loss_list = []

        for image_batch in dataset:
            gen_loss, disc_loss = train_step_gan(image_batch)
            gen_loss_list.append(gen_loss)
            disc_loss_list.append(disc_loss)

        print(f'Epoch {epoch + 1}, Gen Loss: {np.mean(gen_loss_list):.4f}, Disc Loss: {np.mean(disc_loss_list):.4f}, Time: {time.time()-start:.2f} sec')

def train_diffusion(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        loss_list = []

        for image_batch in dataset:
            loss = train_step_diffusion(image_batch)
            loss_list.append(loss)

        print(f'Epoch {epoch + 1}, Loss: {np.mean(loss_list):.4f}, Time: {time.time()-start:.2f} sec')

# Generate images
def generate_and_save_images(model, epoch, test_input, model_name):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, 0] * 0.5 + 0.5, cmap='gray')
        plt.axis('off')

    plt.savefig(f'generated_images/{model_name}_epoch_{epoch:04d}.png')
    plt.close()

# Train models
EPOCHS = 15
print("Training DCGAN...")
train_gan(train_dataset, EPOCHS)
print("\nTraining Diffusion Model...")
train_diffusion(train_dataset, EPOCHS)

# Generate and save images
noise = tf.random.normal([16, 100])
generate_and_save_images(generator, EPOCHS, noise, "dcgan")

noisy_samples = train_images[:16] + 0.1 * tf.random.normal(shape=(16, 28, 28, 1))
generate_and_save_images(diffusion_model, EPOCHS, noisy_samples, "diffusion")





Loading Fashion-MNIST dataset...
29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 2us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


c:\Users\Isaac\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Isaac\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training DCGAN...
Epoch 1, Gen Loss: 0.8797, Disc Loss: 1.0650, Time: 74.05 sec
Epoch 2, Gen Loss: 0.9442, Disc Loss: 1.1992, Time: 128.00 sec
Epoch 3, Gen Loss: 0.9004, Disc Loss: 1.2592, Time: 150.26 sec
Epoch 4, Gen Loss: 0.8581, Disc Loss: 1.2715, Time: 98.43 sec
Epoch 5, Gen Loss: 0.8106, Disc Loss: 1.3245, Time: 75.63 sec
Epoch 6, Gen Loss: 0.8626, Disc Loss: 1.2714, Time: 75.60 sec
Epoch 7, Gen Loss: 0.8691, Disc Loss: 1.2816, Time: 76.12 sec
Epoch 8, Gen Loss: 0.8803, Disc Loss: 1.2718, Time: 78.94 sec
Epoch 9, Gen Loss: 0.8775, Disc Loss: 1.2470, Time: 80.26 sec
Epoch 10, Gen Loss: 0.8819, Disc Loss: 1.2039, Time: 77.20 sec
Epoch 11, Gen Loss: 1.0022, Disc Loss: 1.1838, Time: 75.13 sec
Epoch 12, Gen Loss: 0.9578, Disc Loss: 1.1583, Time: 75.08 sec
Epoch 13, Gen Loss: 1.0315, Disc Loss: 1.1394, Time: 76.00 sec
Epoch 14, Gen Loss: 1.0864, Disc Loss: 1.0272, Time: 76.36 sec
Epoch 15, Gen Loss: 1.1537, Disc Loss: 1.0690, Time: 151.90 sec

Training Diffusion Model...
Epoch 1, Loss:

AttributeError: 'Dense' object has no attribute 'input_shape'